In [1]:
import argparse
import json
from glob import glob
import pandas as pd
import pickle
import numpy as np
import re
import os
import operator as op
import warnings 
from owlready2 import * #
import random
import unicodedata
warnings.filterwarnings('ignore')

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## Carregar ontologia

In [2]:
onto_name = "OntoGeoLogicaInstanciasRelacoes"
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

## Funções para procurar na ontologia

In [3]:
def get_relations_between_uris(uri_1, uri_2): 
    #funcao que acessa a ontologia e procura relacao entre URIs
    dict_relation_uris = {}
    #Pega as relacoes que a URI1 tem
    relation_query_results = list(default_world.sparql("""
            SELECT DISTINCT ?rel
            WHERE{?uri ?rel ?obj
                 FILTER(contains(str(?rel), "http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#"))
                 FILTER (contains(str(?uri), """ + '"' + uri_1 + '"' + """))
                 }
            """))
    
    relations_str = []
    for relation_uris in relation_query_results:
        relations_str.append(str(relation_uris[0]).rsplit(".",1)[-1])
        
    # Para cada tipo de relação procura se existe match entre URI1 e URI2
    for relation in relations_str:
        relation_between_words = list(default_world.sparql("""
                PREFIX prefix: <http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#>
                SELECT distinct ?y ?x2
                WHERE{?y prefix:""" +  relation  +  """ ?x1

                      FILTER (contains(str(?y), """ + '"' + uri_1  + '"' + """))        

                      ?x2 rdf:type ?j                                   
                      FILTER (contains(str(?x2), """ + '"' + uri_2  + '"' + """))

                      FILTER ( ?x2 = ?x1 )
                    }
                """))
        dict_relation_uris[relation] = relation_between_words
    return dict_relation_uris

def go_through_relations(uri1,uri2):
    relation_uris = get_relations_between_uris(uri1, uri2)            
    if relation_uris != {}: #talvez exista relacao entre URIs, dicionario pode vir vazio -> []
        for x, y in relation_uris.items():#procurar por relacao
            if y != []: #existe alguma relacao
#                 print(x)
                return x
    return None

### Funções para printar informações

In [4]:
def print_sentence_text(sentence):
    #printa e retorna o texto original da sentenca
    size_sentence = int(sentence.iloc[-1]["end"])
    text = " "*size_sentence
    for index, row in sentence.iterrows():
        text = text[:int(row["start"])] + row["form"] +text[int(row["end"]):]
    print(text)
    print("-------------")
    return text

def print_relation_entities(word1,word2,ent1,ent2,URI_1,URI_2,relation_type,text):
    #printa as entidades e relacao entre elas
    print('Token 1 = ', word1, '--- Class 1 = ', ent1, '--- URI 1 = ', URI_1)
    print('Token 2 = ', word2, '--- Class 2 = ', ent2,'--- URI 2 = ',URI_2)
    print('Relation Type = ', relation_type)
    print(text)
    print("-------------")
    pass

## Funções para gerar Jsons a serem lidos no labelstudio

In [5]:
class ResultRelationJson(object):
    def __init__(self, from_id, to_id, relations, direction = "right"):
        self.dict = {
            "from_id": str(from_id),
            "to_id": str(to_id),
            "type": "relation",
            "direction": direction,
            "labels": relations
        }
    def get_dict(self):
        return self.dict
class ResultNERJson(object):
    def __init__(self, row):     
        self.result_dict = {
            "value": {
            "start": row["start_word"],
            "end": row["end_word"],
            "text": row["word_join"],
            "labels": [
              row["label_word"]
            ],
            "URI": row["URI"]
            },
            
            "id": row["index_e"],
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "origin": "prediction"
        }
    def get_dict(self):
        return self.result_dict  
class CreateOutput(object):
    def __init__(self, text, filtred_sentence, entity_name_new):
        self.filtred_sentence = filtred_sentence
        self.entity_name_new = entity_name_new
        self.main_dict = {
            "id": 1,
            "data": {
              "text": text #sentenca inteira
            },
            "annotations": []
        }
        self._add_annotations()      
    def _add_annotations(self):
        results = []
        count = 0        
        for index, row in self.entity_name_new.iterrows(): 
            results.append(ResultNERJson(row).get_dict())        
        item = [{
              "id": 1,
              "created_username": " null, 0",
              "created_ago": "",
              "result": results
            }]
        self.main_dict["annotations"] = item
    def get_output(self):
        return self.main_dict
    def add_relationship(self, from_id, to_id, relations, direction):
        results = self.main_dict.get("annotations")[0].get("result")
        relation = ResultRelationJson(from_id, to_id, [relations], direction).get_dict()
        results.append(relation)
        self.main_dict["annotations"][0]["result"] = results   
        
def combine_itens_from_lists_add_in_json(from_id_vec, to_id_vec, relation_from_vec, output):
    for idxRelation in range(0,len(from_id_vec)):
        direction = "right"
        output.add_relationship(from_id=from_id_vec[idxRelation], to_id=to_id_vec[idxRelation], relations = relation_from_vec[idxRelation], direction=direction)
    return output
def saveJsonFiles(df,from_id,to_id, lista_relacoes_sentence,sentence,SentenceNum,path):
    #cria e salva o arquivo Json para labelstudio
    text = sentence.iloc[0]['text']
    print('Saved Json ->', True)
    output = CreateOutput(text,sentence, df)
    combine_itens_from_lists_add_in_json(from_id, to_id, lista_relacoes_sentence, output)
    print("-------------")
    with open(os.path.join(path,f"{SentenceNum}.json"), "w") as outfile: 
        json.dump(output.get_output(), outfile) 
        
def get_df_forJsons(sentence,idxTokens):
    #retorna um dataframe com as informações das entidades e uma string contendo o nome completo da entidade
    df_save_words = pd.DataFrame(columns=['index_e', "LABEL", "START", "END",\
                                      "TEXT", "word_join", "start_word", "end_word", "label_word","URI"])

    index_e = sentence.iloc[idxTokens]['index_e']
    label = sentence.iloc[idxTokens]['deps']
    start = int(sentence.iloc[idxTokens]['word_join_start'])
    end = start + len(sentence.iloc[idxTokens]['form'])
    text_ent = sentence.iloc[idxTokens]['form']
    word_join = sentence.iloc[idxTokens]['word_join']
    start_word = sentence.iloc[idxTokens]['word_join_start']
    end_word = sentence.iloc[idxTokens]['word_join_end']
    label_word = label.replace("B=","")
    URI = sentence.iloc[idxTokens]['grafo']

    df_save_words.loc[len(df_save_words.index)] = [index_e, label, start, end, text_ent, word_join,
                                                   start_word,
                                                   end_word,
                                                   label_word,
                                                   URI]

    return df_save_words, word_join
        
def create_df_JsonFiles(df_entity,x,token,token2,URI_1,URI_2,idxTokens,idxTokens2,from_id,to_id,sentence):
    #retorna o dataframe utilizado para criacao dos arquivos Json para labelstudio
    entity_name_new_token1,wordjoin_1 = get_df_forJsons(sentence,idxTokens)
    entity_name_new_token2,wordjoin_2 = get_df_forJsons(sentence,idxTokens2)
    if idxTokens not in from_id and idxTokens not in to_id:
        df_entity = pd.concat([df_entity, entity_name_new_token1])
    if idxTokens2 not in from_id and idxTokens2 not in to_id:
        df_entity = pd.concat([df_entity, entity_name_new_token2])

#     print_relation_entities(wordjoin_1,wordjoin_2,token.replace('B=',''),token2.replace('B=',''),URI_1,URI_2,x)
    return df_entity  

### Funções para criar dataframe para modelo BERT

In [6]:
def create_moddedText_BERT(text,start1,end1,start2,end2,Ent1_inic,Ent1_end,Ent2_inic,Ent2_end):
    new_end_ent1 = int(end1) + len(Ent1_inic)
    new_start_ent2 = int(start2) + len(Ent1_inic) + len(Ent1_end)
    new_end_ent2 = int(end2) + len(Ent1_inic) + len(Ent1_end) + len(Ent2_inic)      
    #adicionando [E1] e [/E1]
    text_new = text[:int(start1)] + Ent1_inic + text[int(start1):]
    text_new = text_new[:new_end_ent1] + Ent1_end + text_new[new_end_ent1:]
    #adicionando [E2] e [/E2]
    text_new2 = text_new[:new_start_ent2] + Ent2_inic + text_new[new_start_ent2:]
    text_new2 = text_new2[:new_end_ent2] + Ent2_end + text_new2[new_end_ent2:]
    
    return text_new2

def createText_sentence_BERT(text,start_1,end_1,start_2,end_2):
    #funcao que retorna um novo texto para sentenca com [E1] e [E2] adicionados junto de cada entidade
    start_ent1, start_ent2 = start_1, start_2
    end_ent1, end_ent2 = end_1, end_2
    Ent1_inic, Ent1_end = '[E1] ', ' [/E1]'
    Ent2_inic, Ent2_end = '[E2] ', ' [/E2]'
    
    if start_ent1 < start_ent2: #[E1] vem antes de [E2]
        text_new = create_moddedText_BERT(text,start_ent1,end_ent1,start_ent2,end_ent2,\
                                  Ent1_inic,Ent1_end,Ent2_inic,Ent2_end)
    else: #[E2] vem antes de [E1]      
        text_new = create_moddedText_BERT(text,start_ent2,end_ent2,start_ent1,end_ent1,\
                                      Ent2_inic,Ent2_end,Ent1_inic,Ent1_end)  
    return text_new

def create_bert_dataframe(df_bert,idxTokens,idxTokens2,sentence,URI_1,URI_2,has_relation,relation_type,SentenceNumber):
    #retorna o dataframe com as informacoes de cada sentenca para utilizar no modelo BERT
    df_bert_temp = pd.DataFrame(columns=['#Sentence','sentence','Ent1','Ent2','URI_1','URI_2','has_relation','relation'])
    text = sentence.iloc[0]['text']
    wordjoin_1, wordjoin_2 = sentence.iloc[idxTokens]['word_join'], sentence.iloc[idxTokens2]['word_join']
    ent1, ent2 = sentence.iloc[idxTokens]['deps'], sentence.iloc[idxTokens2]['deps']
    ent1, ent2 = ent1.replace("B=",""), ent2.replace("B=","")
    start_1, start_2 = sentence.iloc[idxTokens]['word_join_start'], sentence.iloc[idxTokens2]['word_join_start']
    end_1, end_2 = sentence.iloc[idxTokens]['word_join_end'], sentence.iloc[idxTokens2]['word_join_end']
    text_bert_ents = createText_sentence_BERT(text,start_1,end_1,start_2,end_2)
    df_bert_temp.loc[0] = [SentenceNumber,
                           text_bert_ents,
                           ent1,
                           ent2,
                           URI_1,
                           URI_2,
                           has_relation,
                           relation_type]
    df_bert = pd.concat([df_bert, df_bert_temp])
    if relation_type!='no_relation':
        print_relation_entities(wordjoin_1,wordjoin_2,ent1,ent2,URI_1,URI_2,relation_type,text_bert_ents)
    return df_bert

## Funções para processar as sentenças

In [7]:
#utils
def create_relations_dataframe(df_relation,token,token2,URI_1,URI_2,x,originalSentenceNumber):
    #retorna dataframe das entidades e suas relações em cada linha
    #importante para contabilizar os tipos de relação
    df_relation_new = pd.DataFrame(columns=['Relation','Ent1','Ent2','URI_1','URI_2','#Sentence'])
    df_relation_new.loc[0] = [x,
                            token.replace('B=',''),
                            token2.replace('B=',''),
                            URI_1,
                            URI_2,
                            originalSentenceNumber]
    df_relation = pd.concat([df_relation, df_relation_new])
    return df_relation

def verifica_pares_entidade_interesse(ENT_1, ENT_2,relation_type):
    #verifica se a relacao encontrada vai ser do tipo temporal_relation CRONO->CRONO
    #funcao talvez precise ser atualizada no futuro conforme a ontologia for povoada
    lista_from = ['POÇO','UNIDADE_LITO','UNIDADE_LITO','CAMPO','POÇO','POÇO','UNIDADE_LITO','UNIDADE_LITO']
    lista_to = ['UNIDADE_LITO','NÂOCONSOLID','ROCHA','BACIA','BACIA','CAMPO','BACIA','UNIDADE_CRONO']        
    for idx in range(0,len(lista_to)):
        if lista_from[idx] == ENT_1 and lista_to[idx] == ENT_2:
            return relation_type
    return 'temporal_relation'

#def go_through_sentence(sentence_df, df_relation,df_bert,sent_numb):
def go_through_sentence(sentence_df, df_relation, df_bert, originalSentenceNumber): 
    #percorre a sentenca em busca de relacoes entre entidades anotadas com URIs
    df_entity = pd.DataFrame()
    from_id, to_id = [], []
    relation_from, relation_to = [], []
    lista_relacoes_sentence = []
    lista_relacoes, lista_uris, lista_classes = [], [], []
    
    is_to_save = False
#     df_bert.to_csv(save_csv_name, encoding='utf-8',index=False)
#     df_relation.to_csv('df_relation.csv', encoding='utf-8',index=False)
    for idxTokens in range(len(sentence_df)):
        token, URI_1 = sentence_df.iloc[idxTokens]['deps'], sentence_df.iloc[idxTokens]['grafo']
        for idxTokens2 in range(len(sentence_df)):
            if idxTokens != idxTokens2:
                token2, URI_2 = sentence_df.iloc[idxTokens2]['deps'], sentence_df.iloc[idxTokens2]['grafo']
                has_relation = False
                relation_type = go_through_relations(URI_1,URI_2)
                if relation_type: 
                    print("-------------")
                    print('sentence =', originalSentenceNumber)
                    is_to_save = True
                    has_relation = True
                    Ent1, Ent2 = token.replace("B=",""), token2.replace("B=","")
                    relation_type = verifica_pares_entidade_interesse(Ent1,Ent2,relation_type)
                    lista_relacoes_sentence.append(relation_type)

                    #criar df_bert para BERT RE com codigo do Fabio
                    df_bert = create_bert_dataframe(df_bert,idxTokens,idxTokens2,sentence_df,
                                                    URI_1,URI_2,
                                                    has_relation,relation_type,originalSentenceNumber)

                    #para contabilizar os pares de entidade por relacao
                    df_relation = create_relations_dataframe(df_relation,token,token2,
                                                             URI_1,URI_2,relation_type,originalSentenceNumber)
                    #listas para contabilizar relacoes, uris e classes
                    lista_relacoes.append(relation_type)
                    lista_uris.append(URI_1)
                    lista_uris.append(URI_2)         
                    lista_classes.append(Ent1)
                    lista_classes.append(Ent2)

                    if is_to_createJsons: #se quiser criar Jsons para LabelStudio
                        df_entity = create_df_JsonFiles(df_entity,relation_type,token,token2,URI_1,URI_2,
                                                        idxTokens,idxTokens2,from_id,to_id,sentence_df)
                        from_id.append(idxTokens)
                        to_id.append(idxTokens2) 

                else: #nao achou relacao
                    relation_type = 'no_relation'

                    df_bert = create_bert_dataframe(df_bert,idxTokens,idxTokens2,sentence_df,
                                                    URI_1,URI_2,has_relation,relation_type,originalSentenceNumber)
                        
    return lista_relacoes,lista_uris,lista_classes,\
            df_bert, df_relation, df_entity, \
            lista_relacoes_sentence, from_id, to_id, is_to_save

### Ler arquivo csv (ou pkl) com as sentenças pós filtragem

In [8]:
# Treino
df_filtred_sentences_treino = pickle.load(open('df_filtred_petroner_uri_treino.conllu.pkl', 'rb'))
#df_filtred_sentences_treino = pd.read_csv('df_filtred_petroner_uri_treino_conllu.csv')

df_group_treino = df_filtred_sentences_treino.groupby('sentence')
print('Numero total de sentenças de treino pos-filtragem -> ',len(df_group_treino))

# Valid
df_filtred_sentences_valid = pickle.load(open('df_filtred_petroner_uri_valid.conllu.pkl', 'rb'))
#df_filtred_sentences_valid = pd.read_csv('df_filtred_petroner_uri_valid_conllu.csv')

df_group_valid = df_filtred_sentences_valid.groupby('sentence')
print('Numero total de sentenças de validação pos-filtragem -> ',len(df_group_valid))

# Teste
df_filtred_sentences_teste = pickle.load(open('df_filtred_petroner_uri_teste.conllu.pkl', 'rb'))
#df_filtred_sentences_teste = pd.read_csv('df_filtred_petroner_uri_teste_conllu.csv')

df_group_teste = df_filtred_sentences_teste.groupby('sentence')
print('Numero total de sentenças de teste pos-filtragem -> ',len(df_group_teste))

Numero total de sentenças de treino pos-filtragem ->  3376
Numero total de sentenças de validação pos-filtragem ->  446
Numero total de sentenças de teste pos-filtragem ->  720


### Escolher se deseja criar Jsons para labelstudio

In [9]:
is_to_createJsons = True
# is_to_createJsons = False

### Folder outputs

In [10]:
#save_folder_path = "./JSONs_04_05" #local onde são salvos os Jsons para labelstudio
save_folder_path_treino = "./JSONs_treino"
save_csv_name_bert_treino = 'df_bert_sentences_treino.csv'
save_csv_name_relation_treino = 'df_relation_treino.csv'
lista_relacoes_treino = 'lista_relacoes_treino.pkl'
lista_uris_treino = 'lista_uris_treino.pkl'
lista_classes_treino = 'lista_classes_treino.pkl'

save_folder_path_valid = "./JSONs_valid"
save_csv_name_bert_valid = 'df_bert_sentences_valid.csv'
save_csv_name_relation_valid = 'df_relation_valid.csv'
lista_relacoes_valid = 'lista_relacoes_valid.pkl'
lista_uris_valid = 'lista_uris_valid.pkl'
lista_classes_valid = 'lista_classes_valid.pkl'

save_folder_path_teste = "./JSONs_teste"
save_csv_name_bert_teste = 'df_bert_sentences_teste.csv'
save_csv_name_relation_teste = 'df_relation_teste.csv'
lista_relacoes_teste = 'lista_relacoes_teste.pkl'
lista_uris_teste = 'lista_uris_teste.pkl'
lista_classes_teste = 'lista_classes_teste.pkl'

## Rotina para processar as sentenças já filtradas 

In [11]:
def processar_sentencas_filtradas(df_filtred_sentences, 
                                  df_group, 
                                  save_csv_name_bert, 
                                  save_csv_name_relation, 
                                  save_folder_path, 
                                  lista_relacoes_file, 
                                  lista_uris_file, 
                                  lista_classes_file):

    #%%time

    numberSentences = df_filtred_sentences.iloc[-1]['sentence'] #numero de sentencas diferentes no arquivo ja filtrado
    lista_relacoes, lista_uris, lista_classes, list_sentences_dict = [], [], [], []
    df_relation, df_bert = pd.DataFrame(), pd.DataFrame()

    countJsons=0
    for idx in range(1,len(df_group)):
        filtred_sentence = df_group.get_group(idx)#aqui filtred_sentence é um dataframe da sentenca
        originalSentenceNumber = filtred_sentence.iloc[0]['#sentence_original']
        text = filtred_sentence.iloc[0]['text']
        new_relacoes,new_uris,new_classes,df_bert,\
        df_relation,df_entity,lista_relacoes_sentence,\
        from_id,to_id, is_to_save = go_through_sentence(filtred_sentence,df_relation,df_bert,originalSentenceNumber) 
                                    #go_through_sentence(sentence_df,df_relation,df_bert,sent_numb)
        
        lista_relacoes = lista_relacoes + new_relacoes
        lista_uris = lista_uris + new_uris
        lista_classes = lista_classes + new_classes
        
        df_bert.to_csv(save_csv_name_bert, encoding='utf-8',index=False)
        df_relation.to_csv(save_csv_name_relation, encoding='utf-8',index=False)
        if is_to_save and is_to_createJsons:#Jsons somente criados para sentencas com relacao
            countJsons+=1
            saveJsonFiles(df_entity,from_id,to_id, 
                          lista_relacoes_sentence,filtred_sentence,originalSentenceNumber,save_folder_path)
                
    print("-------------")
    print("Number of Jsons saved = ", countJsons)

    # pickle.dump(df_relation, open('df_relation.pkl','wb'), protocol=pickle.HIGHEST_PROTOCOL)
    df_relation.to_csv(save_csv_name_relation, encoding='utf-8',index=False)
    df_bert.to_csv(save_csv_name_bert, encoding='utf-8',index=False)

    relacoes = np.unique(lista_relacoes, return_counts = True)
    pickle.dump(relacoes, open(lista_relacoes_file,'wb'),protocol = pickle.HIGHEST_PROTOCOL)

    uris = np.unique(lista_uris, return_counts = True)
    pickle.dump(uris,open(lista_uris_file,'wb'),protocol = pickle.HIGHEST_PROTOCOL)

    classes = np.unique(lista_classes, return_counts = True)
    pickle.dump(classes,open(lista_classes_file,'wb'),protocol = pickle.HIGHEST_PROTOCOL)
    
    return

In [22]:
processar_sentencas_filtradas(df_filtred_sentences_treino, 
                              df_group_treino, 
                              save_csv_name_bert_treino, 
                              save_csv_name_relation_treino, 
                              save_folder_path_treino,
                              lista_relacoes_treino, 
                              lista_uris_treino, 
                              lista_classes_treino)

-------------
sentence = 2
Token 1 =   Campo de Curimã --- Class 1 =  CAMPO --- URI 1 =  #CAMP_CD_CAMPO_0352
Token 2 =   Bacia Ceará --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_096
Relation Type =  located_in
Quando aplicados    [E1] Campo de Curimã [/E1] — localizado    [E2] Bacia    Ceará [/E2] e produtor de óleo em arenitos flúvio-deltaicos de idade aptiana — permitiram melhor compreensão    distribuição lateral e vertical     fácies permoporosas e, consequentemente, uma caracterização mais precisa de suas propriedades médias..
-------------
Saved Json -> True
-------------
-------------
sentence = 16
Token 1 =   Campo de Curimã --- Class 1 =  CAMPO --- URI 1 =  #CAMP_CD_CAMPO_0352
Token 2 =   Ceará --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_096
Relation Type =  located_in
O [E1] Campo de Curimã [/E1]                plataforma continental    [E2] Ceará [/E2],    porção central    Sub-bacia de Mundaú, a 75 km de Fortaleza e 50 km    linha de costa, em lâmina d'água de 50 m

* Owlready2 * Warning: ignoring cyclic type of, involving storid 439



-------------
sentence = 2167
Token 1 =   formações Ponta Grossa --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_157
Token 2 =   Devoniano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Devonian
Relation Type =  has_age
final    Siluriano ou    Devoniano  [E2]           [/E2] uma fase de sedimentação marinha, representada       folhelhos cinza-escuros     [E1] formações Ponta Grossa [/E1], Pimenteiras, Maecuru e Bokkeweld..
-------------
-------------
sentence = 2167
Token 1 =   formações Ponta Grossa --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_157
Token 2 =   folhelhos --- Class 2 =  ROCHA --- URI 2 =  #shale
Relation Type =  constituted_by
final    Siluriano ou    Devoniano            uma fase de sedimentação marinha, representada       folhelhos c[E2] inza-escu [/E2]ros     [E1] formações Ponta Grossa [/E1], Pimenteiras, Maecuru e Bokkeweld..
-------------
-------------
sentence = 2167
Token 1 =   Maecuru --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_246
Token 2 =   folhe

* Owlready2 * Warning: ignoring cyclic type of, involving storid 603



-------------
sentence = 2240
Token 1 =   neopaleozóicos --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Paleozoic
Token 2 =   paleozóicos --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Paleozoic
Relation Type =  temporal_relation
Aí, a erosão    delgada capa de sedimentos [E1] neopaleozóicos [/E1] e [E2] paleozóicos [/E2] expôs o embasamento em vasta área (fig. 9)..
-------------
-------------
sentence = 2240
Token 1 =   paleozóicos --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Paleozoic
Token 2 =   neopaleozóicos --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Paleozoic
Relation Type =  temporal_relation
Aí, a erosão    delgada capa de sedimentos [E2] neopaleozóicos [/E2] e [E1] paleozóicos [/E1] expôs o embasamento em vasta área (fig. 9)..
-------------
Saved Json -> True
-------------
-------------
sentence = 2265
Token 1 =   Turoniano --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Turonian
Token 2 =   Coniaciano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Coniacian
Relation Type =  temporal_relation
A 

...

-------------
sentence = 16668
Token 1 =   formações Paracuru --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_200
Token 2 =   idade neo-albiana --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Upper_Albian_Subage
Relation Type =  has_age
Em ambos os casos, blocos rotacio-nados    seção aptiana, estruturando rochas     formações P[E1] aracuru e Mundaú,  [/E1]             truncados    topo por discordância angular de [E2] idade neo-albiana [/E2], o que coloca os reservatórios aptianos em contato direto com os folhelhos    seção marinha transgressiva, garantindo a efetividade    retenção de hidrocarbonetos..
-------------
-------------
sentence = 16668
Token 1 =   formações Paracuru --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_200
Token 2 =   aptianos --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Aptian
Relation Type =  has_age
Em ambos os casos, blocos rotacio-nados    seção aptiana, estruturando rochas     formações P[E1] aracuru e Mundaú,  [/E1]             truncados    topo por discor

* Owlready2 * Warning: ignoring cyclic type of, involving storid 396



-------------
sentence = 16697
Token 1 =   Eomioceno --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Burdigalian
Token 2 =   Eomioceno --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Burdigalian
Relation Type =  temporal_relation
O inferior, fortemente pro-gradacional, constituído de sedimentos batiais a neríticos profundos depositados entre o Meso-campaniano e o [E1] Eomioceno [/E1]; e o superior, essencialmente agradacional, constituído de sedimentos de plataforma rasa de alta energia, depositados entre o [E2] Eomioceno [/E2] e o Holoceno (Pessoa Neto,1999)..
-------------
-------------
sentence = 16697
Token 1 =   Eomioceno --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Burdigalian
Token 2 =   Eomioceno --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Burdigalian
Relation Type =  temporal_relation
O inferior, fortemente pro-gradacional, constituído de sedimentos batiais a neríticos profundos depositados entre o Meso-campaniano e o [E2] Eomioceno [/E2]; e o superior, essencialmente agradacional, constit

* Owlready2 * Warning: ignoring cyclic type of, involving storid 514



-------------
sentence = 16712
Token 1 =   Formação Paracuru --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_200
Token 2 =   idade --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Lower_Albian_Subage
Relation Type =  has_age
campo, a porção superior    Formação Pa[E1] racuru, incluindo [/E1] os calcários     Camadas Trairi, foi removida por eventos erosivos pós-rifte, sendo a discordância angular mais antiga         área, datada como de [E2] idade [/E2]           ..
-------------
Saved Json -> True
-------------
-------------
sentence = 16714
Token 1 =   Formação Mundaú --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_296
Token 2 =   arenitos --- Class 2 =  ROCHA --- URI 2 =  #sandstone
Relation Type =  constituted_by
Três     sete zonas de produção    campo são [E2] arenitos [/E2] fluviais    [E1] Formação Mundaú [/E1] e quatro são arenitos deltaicos    Formação Paracuru..
-------------
-------------
sentence = 16714
Token 1 =   Formação Paracuru --- Class 1 =  UNIDADE_LITO --- URI 1 =

In [12]:
processar_sentencas_filtradas(df_filtred_sentences_valid, 
                              df_group_valid, 
                              save_csv_name_bert_valid, 
                              save_csv_name_relation_valid, 
                              save_folder_path_valid,
                              lista_relacoes_valid, 
                              lista_uris_valid, 
                              lista_classes_valid)

-------------
sentence = 34
Token 1 =   poço 7-BRG-12-SE --- Class 1 =  POÇO --- URI 1 =  #POCO_CD_POCO_007553
Token 2 =   Bacia de Sergipe/Alagoas --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_116
Relation Type =  located_in
Observações:  1.a biozona foi originalmente definida em depósitos    Bacia de Se[E2] rgipe/Alagoas (testemunh [/E2]os    [E1] poço 7-BRG-12-SE [/E1]), por Beurlen et al (1987)..
-------------
Saved Json -> True
-------------
-------------
sentence = 47
Token 1 =   Albiano --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Albian
Token 2 =   Aptiano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Aptian
Relation Type =  temporal_relation
Cronoestratigrafia: [E1] Albiano [/E1], podendo, entretanto, englobar parte    [E2] Aptiano [/E2]..
-------------
-------------
sentence = 47
Token 1 =   Aptiano --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Aptian
Token 2 =   Albiano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Albian
Relation Type =  temporal_relation
Cronoestratigrafia: [E2] A

* Owlready2 * Warning: ignoring cyclic type of, involving storid 370



-------------
sentence = 96
Token 1 =   aptianos --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Aptian
Token 2 =   aptianos --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Aptian
Relation Type =  temporal_relation
Esta constatação induz   formulação de algumas hipóteses para explicar o não-registro       última biozona     bacias mencionadas: a) a existência de amplo hiato intra-albiano, correspondente    tempo compreendido      biozona Nannoconus fragilis; b) as condicionantes ambientais    água    mar necessárias para a proliferação e sedimentação (após a morte    organismo) de Nannoconus fragilis não teriam sido atingidas; c) a recristalização sofrida       carbonatos durante a diagénese teria destruído ou dificultaria muito a liberação     nanofósseis    matriz    rocha, impedindo, assim, o reconhecimento    biozona em análises rotineiras; “e d) eventualmente, já em tempos aptianos (?[E1] ), a mar [/E1]gem equatorial receberia influxos de águas oriundas    oceano situado a norte    primitiv

* Owlready2 * Warning: ignoring cyclic type of, involving storid 367



-------------
sentence = 98
Token 1 =   Albiano --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Albian
Token 2 =   intra-albiano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Albian
Relation Type =  temporal_relation
De acordo com os arcabouços bioestratigráficos elaborados    partir de outros grupos fósseis (palinomorfos, principalmente), o [E1] Albiano [/E1],    margem sudeste, é dividido em duas ou mais biozonas e as investigações não sugerem a existência de qualquer hiato [E2] intra-albiano [/E2] de grande expressão geográfica (Azevedo et al. 1987a; Viviers et al. 1986; Oliveira et al. 1993)..
-------------
-------------
sentence = 98
Token 1 =   intra-albiano --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Albian
Token 2 =   Albiano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Albian
Relation Type =  temporal_relation
De acordo com os arcabouços bioestratigráficos elaborados    partir de outros grupos fósseis (palinomorfos, principalmente), o [E2] Albiano [/E2],    margem sudeste, é dividido em 

...

-------------
sentence = 745
Token 1 =   Série Atokan --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Atokan_Age
Token 2 =   Atokan --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Atokan_Age
Relation Type =  temporal_relation
Cabe ressaltar que a Série Lampasan era definida em [E1] Série Atokan [/E1] ([E2] Atokan [/E2] Series), caracterizada      como equivalente   Zona de Fusulinella e incluía camadas presença de Pseudostafella, Ozawainella, Profusulinella, consideradas desmoinesianas,    passo que a Série Fusiella e Fusulinella..
-------------
-------------
sentence = 745
Token 1 =   Série Atokan --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Atokan_Age
Token 2 =   desmoinesianas --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Desmoinesian_Age
Relation Type =  temporal_relation
Cabe ressaltar que a Série Lampasan era definida em Série Atoka[E1] n (Atokan Se [/E1]ries), caracterizada      como equivalente   Zona de Fusulinella e incluía camadas presença de Pseudostafella, Ozawainella, Profusulinella, co

* Owlready2 * Warning: ignoring cyclic type of, involving storid 578



-------------
sentence = 753
Token 1 =   Morrowan --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Morrowan_Age
Token 2 =   Morrowano/Atokano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Morrowan_Age
Relation Type =  temporal_relation
Assim, o aparecimento de Profusulinella não (1940) propôs o uso formal     cinco séries: Morrowan, c[E1] oincide  [/E1]com o limite [E2] Morrowano/Atokano [/E2]..
-------------
-------------
sentence = 753
Token 1 =   Morrowano/Atokano --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Morrowan_Age
Token 2 =   Morrowan --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Morrowan_Age
Relation Type =  temporal_relation
Assim, o aparecimento de Profusulinella não (1940) propôs o uso formal     cinco séries: Morrowan, c[E2] oincide  [/E2]com o limite [E1] Morrowano/Atokano [/E1]..
-------------
Saved Json -> True
-------------
-------------
sentence = 754
Token 1 =   Atokan --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Atokan_Age
Token 2 =   Desmoinesian --- Class 2 =  UNIDADE_CRONO ---

* Owlready2 * Warning: ignoring cyclic type of, involving storid 388



-------------
sentence = 1146
Token 1 =   Grupo Tapajós --- Class 1 =  UNIDADE_LITO --- URI 1 =  #grupo_049
Token 2 =   Pensilvaniano Inferior --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Bashkirian
Relation Type =  has_age
Marinhos    [E1] Grupo Tapajós [/E1], [E2] Pensilvaniano Inferior [/E2] a Médio    Bacia    Amazonas com aplicação de isótopos de Sr e Nd       intervalo..
-------------
-------------
sentence = 1146
Token 1 =   Grupo Tapajós --- Class 1 =  UNIDADE_LITO --- URI 1 =  #grupo_049
Token 2 =   Bacia Amazonas --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_030
Relation Type =  located_in
Marinhos    [E1] Grupo Tapajós [/E1], Pensilvaniano Inferior a Médio    [E2] Bacia    Amazonas [/E2] com aplicação de isótopos de Sr e Nd       intervalo..
-------------
Saved Json -> True
-------------
-------------
sentence = 1165
Token 1 =   Formação Jandaíra --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_158
Token 2 =   Bacia Potiguar --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACI

In [18]:
processar_sentencas_filtradas(df_filtred_sentences_teste, 
                              df_group_teste, 
                              save_csv_name_bert_teste, 
                              save_csv_name_relation_teste, 
                              save_folder_path_teste,
                              lista_relacoes_teste, 
                              lista_uris_teste, 
                              lista_classes_teste)

-------------
sentence = 6
Token 1 =   Membro Mucuri --- Class 1 =  UNIDADE_LITO --- URI 1 =  #membro_010
Token 2 =   Bacia Espirito Santo --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_270
Relation Type =  located_in
[E1] Membro Mucuri [/E1], Eocretáceo    [E2] Bacia    Espirito Santo [/E2]..
-------------
Saved Json -> True
-------------
-------------
sentence = 97
Token 1 =   Formação Pendência --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_319
Token 2 =   Bacia Potiguar --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_100
Relation Type =  located_in
2.2 - Estratigrafia de Sequências    [E1] Formação Pendência [/E1], [E2] Bacia Potiguar [/E2]  Della Favera et al. (1992) analisam a seção sin-rift    Bacia Potiguar,    qual identificaram quatro sequências balizadas por discordâncias e suas concordâncias relativas dentro    Formação Pendência..
-------------
-------------
sentence = 97
Token 1 =   Formação Pendência --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_319
Token 2 =

* Owlready2 * Warning: ignoring cyclic type of, involving storid 671



-------------
sentence = 349
Token 1 =   membros Oiteirinhos --- Class 1 =  UNIDADE_LITO --- URI 1 =  #membro_044
Token 2 =   idade neoaptiana --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Upper_Aptian_Subage
Relation Type =  has_age
Mesmo                 as incertezas     correlações     idades, diversas determinações bicestratigráficas efetuadas    âmbito    PETROBRAS conferem [E2] idade neoaptiana [/E2] (P-270) para as rochas     [E1] membros Oiteirinhos [/E1] e Ibura..
-------------
-------------
sentence = 349
Token 1 =   Ibura --- Class 1 =  UNIDADE_LITO --- URI 1 =  #membro_061
Token 2 =   idade neoaptiana --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Upper_Aptian_Subage
Relation Type =  has_age
Mesmo                 as incertezas     correlações     idades, diversas determinações bicestratigráficas efetuadas    âmbito    PETROBRAS conferem [E2] idade neoaptiana [/E2] (P-270) para as rochas     membros Oiteirinhos e [E1] Ibura [/E1]..
-------------
Saved Json -> True
-------------
---

* Owlready2 * Warning: ignoring cyclic type of, involving storid 610



-------------
sentence = 723
Token 1 =   Pleistoceno --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Pleistocene
Token 2 =   Pleistoceno --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Pleistocene
Relation Type =  temporal_relation
Os testemunhos longos atingiram as subzonas Globigerina calida calida (parte) e Globigerina bermudezi ([E1] Pleistoceno [/E1]) e Globorotalia fimbriata (Holoceno), que se correlacionam com as zonas paleoclimáticas W (parte), X, Y ([E2] Pleistoceno [/E2]) e Z (Holoceno)..
-------------
-------------
sentence = 723
Token 1 =   Pleistoceno --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Pleistocene
Token 2 =   Holoceno --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Holocene
Relation Type =  temporal_relation
Os testemunhos longos atingiram as subzonas Globigerina calida calida (parte) e Globigerina bermudezi ([E1] Pleistoceno [/E1]) e Globorotalia fimbriata (Holoceno), que se correlacionam com as zonas paleoclimáticas W (parte), X, Y (Pleistoceno) e Z ([E2] Holoceno [/E2])..
----

...

-------------
sentence = 2865
Token 1 =   Fm. Riachuelo --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_130
Token 2 =   eocenomaniana --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Lower_Cenomanian_Subage
Relation Type =  has_age
MENDES, J. M. C. Análise estratigráfica    seção neo-aptiniana/[E2] eocenomaniana [/E2] ([E1] Fm. Riachuelo [/E1])    área    Alto de Aracaju e adjacências – Bacia de Sergipe/Alagoas..
-------------
-------------
sentence = 2865
Token 1 =   Fm. Riachuelo --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_130
Token 2 =   Bacia de Sergipe/Alagoas --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_116
Relation Type =  located_in
MENDES, J. M. C. Análise estratigráfica    seção neo-aptiniana/eocenomaniana (Fm. Riachue[E1] lo)    área   [/E1]  Alto de Aracaju e adjacências – [E2] Bacia de Sergipe/Alagoas [/E2]..
-------------
Saved Json -> True
-------------
-------------
sentence = 2877
Token 1 =   Neojurássico --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #UpperJurassic

* Owlready2 * Warning: ignoring cyclic type of, involving storid 658



-------------
sentence = 2920
Token 1 =   Formação Açu --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_253
Token 2 =   Turoniano --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Turonian
Relation Type =  has_age
Aibiano    [E2] Turoniano [/E2],              a [E1] Formação Açu [/E1] (SAMPAIO & SCHALLER, 1968), composta de arenitos flúvio-deltaicos que extravasaram em muito os limites    graben inicial, am-pliando a área    bacia sedimentar e aflorando em sua borda erosiva atual..
-------------
-------------
sentence = 2920
Token 1 =   Formação Açu --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_253
Token 2 =   arenitos --- Class 2 =  ROCHA --- URI 2 =  #sandstone
Relation Type =  constituted_by
Aibiano    Turoniano,              a [E1] Formação Açu [/E1] (SAMPAIO & SCHALLER, 1968), composta de [E2] arenitos [/E2] flúvio-deltaicos que extravasaram em muito os limites    graben inicial, am-pliando a área    bacia sedimentar e aflorando em sua borda erosiva atual..
-------------
Saved Jso

* Owlready2 * Warning: ignoring cyclic type of, involving storid 433



-------------
sentence = 3408
Token 1 =   Formação Macaé --- Class 1 =  UNIDADE_LITO --- URI 1 =  #grupo_000
Token 2 =   cretácea --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Cretaceous
Relation Type =  has_age
A primeira contribuição significativa   Palinologia marinha cretácea   [E2]  Brasil  [/E2]foi lançada em 1976      geólogo Namio Uesugui, que adotou três espécies de dinoflagelados e uma de acritarco para subdividir a [E1] Formação Macaé [/E1]..
-------------
Saved Json -> True
-------------
-------------
sentence = 3427
Token 1 =   Mesozóico --- Class 1 =  UNIDADE_CRONO --- URI 1 =  #Mesozoic
Token 2 =   Paleozóico --- Class 2 =  UNIDADE_CRONO --- URI 2 =  #Paleozoic
Relation Type =  temporal_relation
Isto porque,    [E1] Mesozóico [/E1], os palinomorfos marinhos que prosperaram    [E2] Paleozóico [/E2] se tornaram decadentes e numerica-mente insignificantes – como    caso de acritarcos – ou, senão, totalmente extintos – como     casos de quitinozoários e graptolitos..
----------

### Contabilização das relações encontradas

In [14]:
def contabilizar(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    for i in range(len(data[0])):
        print(data[0][i], ' - ', data[1][i]) 

In [23]:
contabilizar(lista_relacoes_treino)

constituted_by  -  297
crosses  -  15
has_age  -  123
located_in  -  395
temporal_relation  -  752


In [15]:
contabilizar(lista_relacoes_valid)

constituted_by  -  12
crosses  -  8
has_age  -  7
located_in  -  47
temporal_relation  -  145


In [19]:
contabilizar(lista_relacoes_teste)

constituted_by  -  93
crosses  -  6
has_age  -  61
located_in  -  84
temporal_relation  -  247


### Contabilização das classes encontradas

In [24]:
contabilizar(lista_classes_treino)

BACIA  -  380
CAMPO  -  121
NÃOCONSOLID  -  6
POÇO  -  70
ROCHA  -  297
UNIDADE_CRONO  -  1485
UNIDADE_LITO  -  805


In [16]:
contabilizar(lista_classes_valid)

BACIA  -  42
CAMPO  -  15
NÃOCONSOLID  -  1
POÇO  -  25
ROCHA  -  12
UNIDADE_CRONO  -  283
UNIDADE_LITO  -  60


In [20]:
contabilizar(lista_classes_teste)

BACIA  -  70
CAMPO  -  26
POÇO  -  23
ROCHA  -  93
UNIDADE_CRONO  -  479
UNIDADE_LITO  -  291


### Contabilização das URIs encontradas 

In [25]:
contabilizar(lista_uris_treino)

#Alagoas_Age  -  54
#Albian  -  83
#Aptian  -  76
#Aratu_Age  -  19
#Archean  -  6
#Atokan_Age  -  4
#BASE_CD_BACIA_020  -  2
#BASE_CD_BACIA_030  -  1
#BASE_CD_BACIA_051  -  3
#BASE_CD_BACIA_076  -  2
#BASE_CD_BACIA_080  -  1
#BASE_CD_BACIA_090  -  2
#BASE_CD_BACIA_096  -  31
#BASE_CD_BACIA_100  -  42
#BASE_CD_BACIA_106  -  4
#BASE_CD_BACIA_116  -  79
#BASE_CD_BACIA_210  -  1
#BASE_CD_BACIA_215  -  3
#BASE_CD_BACIA_230  -  5
#BASE_CD_BACIA_240  -  35
#BASE_CD_BACIA_250  -  8
#BASE_CD_BACIA_256  -  1
#BASE_CD_BACIA_260  -  8
#BASE_CD_BACIA_266  -  20
#BASE_CD_BACIA_270  -  23
#BASE_CD_BACIA_281  -  55
#BASE_CD_BACIA_300  -  28
#BASE_CD_BACIA_316  -  22
#BASE_CD_BACIA_381  -  4
#Barremian  -  13
#Bartonian  -  2
#Buracica_Age  -  29
#Burdigalian  -  4
#CAMP_CD_CAMPO_0003  -  1
#CAMP_CD_CAMPO_0004  -  1
#CAMP_CD_CAMPO_0012  -  1
#CAMP_CD_CAMPO_0017  -  4
#CAMP_CD_CAMPO_0027  -  1
#CAMP_CD_CAMPO_0065  -  3
#CAMP_CD_CAMPO_0082  -  6
#CAMP_CD_CAMPO_0093  -  1
#CAMP_CD_CAMPO_0118  -  2
#CAMP_

In [17]:
contabilizar(lista_uris_valid)

#Albian  -  24
#Aptian  -  18
#Atokan_Age  -  36
#BASE_CD_BACIA_030  -  10
#BASE_CD_BACIA_096  -  4
#BASE_CD_BACIA_100  -  1
#BASE_CD_BACIA_116  -  8
#BASE_CD_BACIA_240  -  4
#BASE_CD_BACIA_266  -  4
#BASE_CD_BACIA_270  -  3
#BASE_CD_BACIA_281  -  8
#Barremian  -  2
#Bashkirian  -  1
#CAMP_CD_CAMPO_0247  -  1
#CAMP_CD_CAMPO_0444  -  1
#CAMP_CD_CAMPO_0603  -  1
#CAMP_CD_CAMPO_0718  -  1
#CAMP_CD_CAMPO_0750  -  1
#CAMP_CD_CAMPO_0773  -  5
#CAMP_CD_CAMPO_0888  -  3
#CAMP_CD_CAMPO_0932  -  2
#Campanian  -  37
#Carboniferous  -  17
#Cenomanian  -  12
#Coniacian  -  2
#Desmoinesian_Age  -  16
#Holocene  -  16
#LowerCretaceous  -  4
#Maastrichtian  -  12
#Missourian_Age  -  8
#Morrowan_Age  -  20
#POCO_CD_POCO_007020  -  2
#POCO_CD_POCO_007040  -  4
#POCO_CD_POCO_007052  -  1
#POCO_CD_POCO_007076  -  2
#POCO_CD_POCO_007553  -  2
#POCO_CD_POCO_008339  -  1
#POCO_CD_POCO_010471  -  4
#POCO_CD_POCO_012996  -  4
#POCO_CD_POCO_022660  -  2
#POCO_CD_POCO_023241  -  3
#Paleocene  -  2
#Paleogene  - 

In [21]:
contabilizar(lista_uris_teste)

#Alagoas_Age  -  10
#Albian  -  22
#Aptian  -  18
#Aratu_Age  -  12
#BASE_CD_BACIA_030  -  1
#BASE_CD_BACIA_100  -  6
#BASE_CD_BACIA_116  -  24
#BASE_CD_BACIA_215  -  1
#BASE_CD_BACIA_230  -  6
#BASE_CD_BACIA_240  -  16
#BASE_CD_BACIA_260  -  1
#BASE_CD_BACIA_266  -  2
#BASE_CD_BACIA_270  -  1
#BASE_CD_BACIA_281  -  12
#Barremian  -  4
#Berriasian  -  2
#Buracica_Age  -  16
#CAMP_CD_CAMPO_0017  -  1
#CAMP_CD_CAMPO_0065  -  1
#CAMP_CD_CAMPO_0077  -  4
#CAMP_CD_CAMPO_0199  -  1
#CAMP_CD_CAMPO_0214  -  4
#CAMP_CD_CAMPO_0249  -  3
#CAMP_CD_CAMPO_0264  -  2
#CAMP_CD_CAMPO_0682  -  4
#CAMP_CD_CAMPO_0683  -  1
#CAMP_CD_CAMPO_0684  -  1
#CAMP_CD_CAMPO_0773  -  1
#CAMP_CD_CAMPO_0830  -  1
#CAMP_CD_CAMPO_0881  -  1
#CAMP_CD_CAMPO_0958  -  1
#Cambrian  -  4
#Campanian  -  9
#Cenomanian  -  6
#Cenozoic  -  8
#Coniacian  -  2
#Cretaceous  -  17
#Devonian  -  4
#Dom_Joao_Age  -  23
#Eojiquia_Subage  -  2
#Hauterivian  -  4
#Holocene  -  28
#Jiquia_Age  -  21
#Jurassic  -  18
#LowerCretaceous  -  53


### Verificação de pares de entidades por tipo de relação

In [ ]:
# df_filtred = pickle.load(open('df_relation.pkl', 'rb'))
#df_relations = pd.read_csv('df_relation.csv')
#df_grp = df_relations.groupby('Relation')
#relations_groups = df_grp.groups
#relations = list(relations_groups)
#lista_pares = []
#for relation in relations:
#    df_rel = df_grp.get_group(relation)
#    list_rel = []
#    for idx_rel in range(0,len(df_rel)):
#        par = df_rel.iloc[idx_rel]['Ent1'] + ' + ' + df_rel.iloc[idx_rel]['Ent2']
#        list_rel.append(par)
#    lista_pares.append(list_rel)
#print('Number of types of relations ->', len(lista_pares))

### Avaliar idx_pair de 0 ao tamanho apresentado acima para verificar os pares de entidades

In [ ]:
#idx = 0
#pares, numb_pares = np.unique(lista_pares[idx], return_counts = True)
#print('Relation -> ',relations[idx])
#print('Entities pair -> ',pares.tolist())
#print('Number of ocorrences -> ',numb_pares.tolist())